### Resources consulted in the process of making this

https://machinelearningmastery.com/develop-character-based-neural-language-model-keras/

https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

http://www.tensorflow.org/alpha/tutorials/text/text_generation


### To read: Alternate?

https://towardsdatascience.com/the-fall-of-rnn-lstm-2d1594c74ce0

https://jalammar.github.io/illustrated-transformer/

http://nlp.seas.harvard.edu/2018/04/03/attention.html

https://arxiv.org/abs/1803.01271


# Text Generation

The general read in text as a sequence of characters, predict what the next character will be, with a LSTM/RNN
    

In [101]:
import pandas as pd

import numpy as np

import string
import time
import re 
import os

import tensorflow as tf
tf.enable_eager_execution()


import matplotlib.pyplot as plt


In [102]:
from __future__ import absolute_import, division, print_function, unicode_literals


In [103]:

#filename = 'hamlet3.txt'
filename= 'complete_shakespeare.txt'

### Below is the number of characters to read as a single input
numCharacters = 100



fid = open(filename, 'r',encoding='utf-8-sig')
text = fid.read()
fid.close()

In [104]:
len(text)

5653528

In [105]:
text[0:500]

'\n\n\n\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n               THE SONNETS\n\n               ALL’S WELL THAT ENDS WELL\n\n               THE TRAGEDY OF ANTONY AND CLEOPATRA\n\n               AS YOU LIKE IT\n\n               THE COMEDY OF ERRORS\n\n               THE TRAGEDY OF CORIOLANUS\n\n               CYMBELINE\n\n               THE TRAGEDY OF HAMLET, PRINCE OF DENMARK\n\n               THE FIRST PART OF KING HENRY THE FOURTH\n\n               THE SECOND PART OF K'

In [106]:
print(text[10500:11000])

il luck,
Of plagues, of dearths, or seasons’ quality,
Nor can I fortune to brief minutes tell;
Pointing to each his thunder, rain and wind,
Or say with princes if it shall go well
By oft predict that I in heaven find.
But from thine eyes my knowledge I derive,
And constant stars in them I read such art
As truth and beauty shall together thrive
If from thy self, to store thou wouldst convert:
  Or else of thee this I prognosticate,
  Thy end is truth’s and beauty’s doom and date.


              


In [107]:
len(text)

5653528

In [108]:
segments = list()

for i in range(0, len(text)-numCharacters):
    sequence = text[i:i+1+numCharacters]
    segments.append(sequence)

In [109]:
segments[-10:]

['to Paphos, where their queen\n  Means to immure herself and not be seen.\n\n\n    \n\n  FINIS\n    \n\n\n\n\n\n\n\n\n',
 'o Paphos, where their queen\n  Means to immure herself and not be seen.\n\n\n    \n\n  FINIS\n    \n\n\n\n\n\n\n\n\n\n',
 ' Paphos, where their queen\n  Means to immure herself and not be seen.\n\n\n    \n\n  FINIS\n    \n\n\n\n\n\n\n\n\n\n\n',
 'Paphos, where their queen\n  Means to immure herself and not be seen.\n\n\n    \n\n  FINIS\n    \n\n\n\n\n\n\n\n\n\n\n\n',
 'aphos, where their queen\n  Means to immure herself and not be seen.\n\n\n    \n\n  FINIS\n    \n\n\n\n\n\n\n\n\n\n\n\n\n',
 'phos, where their queen\n  Means to immure herself and not be seen.\n\n\n    \n\n  FINIS\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 'hos, where their queen\n  Means to immure herself and not be seen.\n\n\n    \n\n  FINIS\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 'os, where their queen\n  Means to immure herself and not be seen.\n\n\n    \n\n  FINIS\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [110]:
#for m,n in enumerate(unique_chars):
#    print(m,n)

unique_chars = sorted(set(text))

character_integer_map = dict((character, integer) for integer, character in enumerate(unique_chars))

integer_character_map = dict((integer, character) for integer, character in enumerate(unique_chars))


In [111]:
### Exact same as above but whatev
#unique_chars = sorted(set(text))
#character_integer_map ={u:i for i,u in enumerate(unique_chars)}

In [112]:
num_unique = len(character_integer_map)
print(num_unique, 'unique characters found')

93 unique characters found


In [113]:
segments[0:10]

['\n\n\n\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n       ',
 '\n\n\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n        ',
 '\n\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n         ',
 '\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n          ',
 'The Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n           ',
 'he Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n            ',
 'e Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n             ',
 ' Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n              ',
 'Complete Works of William Shakespear

In [114]:
num_total_characters = len(text)

In [115]:
X_train = []
y_train = []
for i in range(0, num_total_characters - numCharacters, 1):
    sequence = text[i:i + numCharacters]
    sequence_out = text[i + numCharacters]
    X_train.append([character_integer_map[x] for x in sequence])
    y_train.append(character_integer_map[sequence_out])


num_patterns = len(X_train)

# Alternate data prep method

As taken from tensorflow tutorial on text generation (see links at top)

In [116]:

text_as_integer = np.array([character_integer_map[x] for x in text])

In [117]:
text_as_integer[200:300]

array([ 2, 26, 39, 29,  2, 28, 37, 30, 40, 41, 26, 45, 43, 26,  1,  1,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 26, 44,  2,
       50, 40, 46,  2, 37, 34, 36, 30,  2, 34, 45,  1,  1,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 45, 33, 30,  2, 28, 40,
       38, 30, 29, 50,  2, 40, 31,  2, 30, 43, 43, 40, 43, 44,  1,  1,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 45])

In [118]:
''.join([integer_character_map[x] for x in text_as_integer[200:300]])

' AND CLEOPATRA\n\n               AS YOU LIKE IT\n\n               THE COMEDY OF ERRORS\n\n               T'

In [119]:
examples_per_epoch = len(text) // numCharacters

character_dataset = tf.data.Dataset.from_tensor_slices(text_as_integer)

In [120]:
sequences = character_dataset.batch(numCharacters+1, drop_remainder=True)

In [121]:
idx2char = np.array(unique_chars)

for i in sequences.take(5):
    print(repr(''.join(idx2char[i.numpy()])))

'\n\n\n\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n       '
'        THE SONNETS\n\n               ALL’S WELL THAT ENDS WELL\n\n               THE TRAGEDY OF ANTONY A'
'ND CLEOPATRA\n\n               AS YOU LIKE IT\n\n               THE COMEDY OF ERRORS\n\n               THE '
'TRAGEDY OF CORIOLANUS\n\n               CYMBELINE\n\n               THE TRAGEDY OF HAMLET, PRINCE OF DENM'
'ARK\n\n               THE FIRST PART OF KING HENRY THE FOURTH\n\n               THE SECOND PART OF KING H'


In [122]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:] # this is everything previous, shifted over by one
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [123]:
for input_example, target_example in dataset.take(5):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Output data: ', repr(''.join(idx2char[input_example.numpy()])))

Input data:  '\n\n\n\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n      '
Output data:  '\n\n\n\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n      '
Input data:  '        THE SONNETS\n\n               ALL’S WELL THAT ENDS WELL\n\n               THE TRAGEDY OF ANTONY '
Output data:  '        THE SONNETS\n\n               ALL’S WELL THAT ENDS WELL\n\n               THE TRAGEDY OF ANTONY '
Input data:  'ND CLEOPATRA\n\n               AS YOU LIKE IT\n\n               THE COMEDY OF ERRORS\n\n               THE'
Output data:  'ND CLEOPATRA\n\n               AS YOU LIKE IT\n\n               THE COMEDY OF ERRORS\n\n               THE'
Input data:  'TRAGEDY OF CORIOLANUS\n\n               CYMBELINE\n\n               THE TRAGEDY OF HAMLET, PRINCE OF DEN'
Output data:  'TRAGEDY OF CORIOLANUS\n\n               CYMBELINE\n\n               THE TRAGEDY OF HAMLET, PRINCE OF DEN'
Input 

In [124]:
batch_size = 64

# Shuffle in data off a buffer, rather than keep entire sequence in memory
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [125]:
embedding_dimension = 256

rnn_units = 512



In [241]:
def build_model(vocab_size, embedding_dimension, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dimension, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


#def build_model(vocab_size, embedding_dimension, rnn_units, batch_size):
#    model = tf.keras.Sequential([
#        tf.keras.layers.Embedding(vocab_size, embedding_dimension, batch_input_shape=[batch_size, None]),
#        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
#        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
#        tf.keras.layers.Dense(vocab_size)
#    ])
#    return model

In [217]:
model = build_model(vocab_size = num_unique, 
                    embedding_dimension=embedding_dimension, 
                    rnn_units=rnn_units, 
                    batch_size=batch_size)

In [218]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 93)


In [219]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (64, None, 256)           23808     
_________________________________________________________________
lstm_16 (LSTM)               (64, None, 512)           1574912   
_________________________________________________________________
lstm_17 (LSTM)               (64, None, 512)           2099200   
_________________________________________________________________
dense_12 (Dense)             (64, None, 93)            47709     
Total params: 3,745,629
Trainable params: 3,745,629
Non-trainable params: 0
_________________________________________________________________


In [220]:
sample_index = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sample_index = tf.squeeze(sample_index, axis=-1).numpy()

In [221]:
sample_index

array([ 9,  8, 32, 66, 49, 25, 75, 64, 81, 43, 89, 56, 85, 70, 37,  4, 83,
       42, 24, 84, 77, 44, 67, 24, 91, 73, 82, 11, 85,  1,  5, 56, 19, 92,
       14, 66, 46, 56, 56, 74,  9, 79,  2, 85,  9, 63, 86, 36, 43, 26, 59,
       39, 85, 45, 88, 64, 29, 33, 13, 67, 72, 28, 43, 28, 20, 80, 58, 80,
        5, 53, 51,  0,  4, 87,  3, 47, 41, 31, 25, 88, 33, 62, 75, 82, 44,
       11, 82, 20, 21, 33, 18, 82,  0, 40, 25, 87, 56, 80, 30,  5])

In [222]:
# Note that for a random/glorot uniform initialization on an untrained network the result is random gibberish

print('Input:')
print(''.join(idx2char[input_example_batch[0]]))

print()
print('Output:')
print(''.join(idx2char[sample_index]))


Input:
   Desir'd more than constrain'd. To satisfy,
    If of my freedom 'tis the main part, take
    No s

Output:
*)GjX?shyR‘`ÆnL"|Q;}uSk;“qz-Æ
&`6”1jU``r*w Æ*gàKRAcNÆT—hDH0kpCRC7xbx&\Z	"æ!VPF?—HfszS-z78H5z	O?æ`xE&


In [223]:
# Define loss function
# At this point it's a simple multiclass classification problem 

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print('Prediction shape: ', example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print('scalar loss: ', example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 93)  # (batch_size, sequence_length, vocab_size)
scalar loss:  4.531937


In [224]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss=loss)



In [225]:
# Set model checkpoints to make sure they're saved
path = './training_checkpoints'

checkpoint_prefix = os.path.join(path, "checkpoint_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only=True)

In [226]:
# call this line only if you want to load previous checkpoints
print(tf.train.latest_checkpoint(path))
model.load_weights(tf.train.latest_checkpoint(path))



./training_checkpoints/checkpoint_9

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent.LSTM object at 0xb5ec65a58> and <tensorflow.python.keras.layers.core.Dense object at 0xb5657cd30>).


In [227]:
numEpochs = 10


history = model.fit(dataset, epochs=numEpochs,steps_per_epoch=100, callbacks=[checkpoint_callback])

Epoch 1/10
100/100 [==============================] - 303s 3s/step - loss: 1.9524
Epoch 2/10
100/100 [==============================] - 300s 3s/step - loss: 1.6110
Epoch 3/10
100/100 [==============================] - 302s 3s/step - loss: 1.5291
Epoch 4/10
100/100 [==============================] - 307s 3s/step - loss: 1.4772
Epoch 5/10
100/100 [==============================] - 297s 3s/step - loss: 1.4210
Epoch 6/10
100/100 [==============================] - 299s 3s/step - loss: 1.4109
Epoch 7/10
100/100 [==============================] - 300s 3s/step - loss: 1.4048
Epoch 8/10
100/100 [==============================] - 286s 3s/step - loss: 1.4215
Epoch 9/10
 74/100 [=====================>........] - ETA: 1:14 - loss: 1.4023

# Text Generation

In [228]:
tf.train.latest_checkpoint(path)

'./training_checkpoints/checkpoint_9'

In [242]:
model = build_model(vocab_size = num_unique, 
                    embedding_dimension=embedding_dimension, 
                    rnn_units=rnn_units, 
                    batch_size=1)


model.load_weights(tf.train.latest_checkpoint(path +'/Single Layer LSTM'))
#model.load_weights('./training_checkpoints/checkpoint_9')

model.build(tf.TensorShape([1, None]))

In [243]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (1, None, 256)            23808     
_________________________________________________________________
lstm_26 (LSTM)               (1, None, 512)            1574912   
_________________________________________________________________
dense_17 (Dense)             (1, None, 93)             47709     
Total params: 1,646,429
Trainable params: 1,646,429
Non-trainable params: 0
_________________________________________________________________


In [244]:

#Note: setting temperature higher leads to more surprising results
# Setting it lower leads to more predictable results
def generate_text(model, start_string, num_generate = 1000, temperature = 1.0):
    input_eval = [character_integer_map[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    generated_text = []
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # Remove batch dimensions
        predictions = tf.squeeze(predictions, 0)
        
        # Use a categorial distribution to predict word returned by model
        predictions = predictions/ temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # Pass predicted word as next input of model, along with previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        generated_text.append(idx2char[predicted_id])
        
    return ''.join(generated_text)



In [247]:
%time print(generate_text(model, 'Once more into the breach'))

eful band.

GRIMON.
My child-bign.

STARDAMUS.
Why, then?

SERVOLIO.
Nay, 'long, and pleading, and so?
Here’s honest me, desire
    In their govern'd, May is o you Note sently sir my helf, be ranger,
    And the earth shade the ring mad shall, we keep the day
    That you was soth some owl wanton's hand;
    Springs, but I'll capint steep, 'emembrest,
    Their hauds she saide.      [MIOLA.
      The worls way unfaviclity ball.
    Afore your villainm's eaga swor from frain.
                                             Exent

                          In another son Ary.
  CAMLIBALIA. Whereine have you see you my lord is thinr Scerts
    Nay from the act and lookely.
  MIRANDARD. So speak;
Haroust to call hims to other; where ne'er his very doth come quoth me, my lord.
IS was herselfes are they love so;
And go, that I know a kind in ither, 'tis; but itle I have was all with to him for
Where gavis juce to the heart with made part in atsel:
  DIO. That's not will desides thy farbanly arp

# Older code below

# Data transformation

To do: 

1. Get data into format: [samples, time steps, features]

2. Normalize X_train to be between 0 and 1

3. Turn the output vector Y_train to be categorical variables using one-hot encoding


In [81]:
X_train = np.reshape(X_train, (num_patterns , numCharacters, 1))

In [82]:
X_train = X_train / num_unique

# Keras

In [104]:
from keras.utils import np_utils

y = np_utils.to_categorical(y_train)

In [84]:
X_train.shape

(190407, 50, 1)

In [105]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [108]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

model = Sequential()
model.add(LSTM(256, input_shape =( X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True))
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


filepath="weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X_train, y, epochs=20, batch_size=128, callbacks=callbacks_list)


Epoch 1/20
 26368/190407 [===>..........................] - ETA: 15:26 - loss: 3.0961

KeyboardInterrupt: 

# Using trained model to generate text

In [ ]:
#filename = ''
#model.load_weights(filename)

pattern = X_train[np.random.randint(0,len(X_train)-1)]

In [ ]:
numToGenerate = 2000

for i in range(numToGenerate):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = integer_character_map[index]
    seq_in = [integer_character_map[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

# Pytorch

following links were helpful for references debugging

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

http://www.jessicayung.com/lstms-for-time-series-in-pytorch/

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1, num_layers = 2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        
        self.linear = nn.Linear(self.hidden_dim, output_dim)
        
    def init_hidden(self):
        return(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim), 
               torch.zeros(self.num_layers,self.batch_size, self.hidden_dim))
    
    def forward(self, input):
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)
    

        

In [50]:
#X_train = torch.from_numpy(X_train).type(torch.Tensor)


In [51]:
#numEpochs = 50
#batch_size = 50
#learning_rate = 0.1
#hidden_dimension = 3
#
#model = LSTM(numCharacters, hidden_dimension, batch_size, 1, 3)
#
#loss_fn = torch.nn.MSELoss(size_average=False)
#
#optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
#
#performance_data  = np.zeros(numEpochs)
#
#for i in range(numEpochs):
#    model.zero_grad()
#    
#    model.hidden = model.init_hidden()
#    y_pred = model(X_train)
#    optimiser.zero_grad()
#    
#    loss.backward()
#    
#    optimiser.step()

/Users/rcheung/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


RuntimeError: input.size(-1) must be equal to input_size. Expected 50, got 1